### CP2 - Relatório com Análise dos Dados Automatizada

**Integrantes**
- **Lucas Laia Manentti** - RM: 97709
- **Rony Ken Nagai** - RM: 551549
- **Tomáz Versolato Carballo** - RM: 551417

### Objetivo do Trabalho

O objetivo deste trabalho é desenvolver um sistema automatizado para análise de dados, utilizando Python e bibliotecas populares como **pandas**, **matplotlib** e **seaborn**, além de empregar o **jinja2** para a geração de relatórios HTML estilizados com **Bootstrap**. O sistema recebe um dataset e gera automaticamente um relatório com as principais estatísticas e gráficos, facilitando a análise exploratória e a interpretação dos dados.

### Estrutura do Código

#### Bibliotecas Utilizadas

- **pandas**: Manipulação e análise dos dados em formato de dataframes.
- **matplotlib** e **seaborn**: Geração de gráficos para visualização dos dados.
- **jinja2**: Motor de templates para renderizar o HTML do relatório.
- **os**, **base64** e **BytesIO**: Manipulação de arquivos e conversão de gráficos para Base64 para inclusão no HTML.

#### Passo a Passo do Código

1. **Configuração do Template HTML:**
   - O código cria um template HTML utilizando **Bootstrap** para estilização. O template inclui cards e seções bem estruturadas para mostrar as estatísticas e gráficos gerados, garantindo uma apresentação clara e organizada das informações.

2. **Função `resumo_df(df)`:**
   - Calcula as estatísticas principais do dataset, como:
     - Número de variáveis.
     - Número de observações.
     - Células faltantes.
     - Linhas duplicadas.
     - Quantidade de variáveis numéricas e categóricas.
   - Retorna essas informações em um dicionário chamado `summary`.

3. **Função `gerar_conclusao(df)`:**
   - Analisa as correlações, outliers e distribuições das variáveis numéricas e gera conclusões automáticas com base nos dados.
   - As conclusões são apresentadas em uma seção estruturada no relatório HTML.

4. **Função `plot_64(fig)`:**
   - Converte os gráficos gerados em uma string Base64 para que possam ser incorporados no HTML diretamente. Isso garante que os gráficos sejam exibidos corretamente independentemente do ambiente.

5. **Função `criar_plot(df)`:**
   - Gera diversos tipos de gráficos para as variáveis numéricas e categóricas, incluindo:
     - Histogramas
     - Gráficos de dispersão
     - Regressão linear
     - Boxplots
     - Mapas de calor
   - Utiliza **seaborn** para estilizar os gráficos e **matplotlib** para renderizá-los.

6. **Função `gerar_html(df)`:**
   - Renderiza o template HTML utilizando os dados e gráficos gerados pelas funções anteriores.
   - Insere as estatísticas, gráficos e conclusões no relatório e salva o arquivo HTML no ambiente do Colab.

7. **Execução Principal (`if __name__ == '__main__'`):**
   - Carrega o dataset (`Diabetes.csv`) e gera o relatório HTML utilizando as funções definidas.
   - Exibe o relatório diretamente no notebook do Colab utilizando **IFrame** para visualização.

### Estrutura do Relatório

O relatório HTML gerado possui as seguintes seções:

1. **Título e Sumário:**
   - Exibe informações básicas como o número de linhas, colunas, variáveis, observações, células faltantes e linhas duplicadas, com descrições claras e acessíveis para leigos.

2. **Tipos de Dados:**
   - Lista os tipos de dados presentes em cada coluna, explicando a estrutura das variáveis.

3. **Estatísticas Descritivas:**
   - Mostra medidas como média, mediana, desvio padrão, valores mínimos e máximos para cada variável numérica. Cada estatística é detalhada para que o usuário entenda seu significado e importância.

4. **Valores Faltantes:**
   - Mostra a quantidade de valores faltantes em cada coluna e a importância de tratá-los para garantir a qualidade dos dados.

5. **Gráficos:**
   - Exibe diversos tipos de gráficos exploratórios, incluindo histogramas, gráficos de dispersão, regressões lineares, boxplots e mapas de calor. Cada gráfico é explicado para que até pessoas sem conhecimento técnico possam entender as informações visuais.

6. **Conclusões das Análises:**
   - Apresenta uma lista de conclusões automáticas com base nas análises feitas, como correlações significativas, presença de outliers e distribuições assimétricas.

7. **Considerações Finais:**
   - Explica o propósito geral do projeto, as metodologias utilizadas e os principais achados, sugerindo possíveis próximas etapas para aprofundar as análises.

### Como Executar

1. Certifique-se de que todas as bibliotecas necessárias estão instaladas no ambiente Colab:
   ```python
   !pip install jinja2
   !pip install matplotlib seaborn pandas
   ```

2. Execute o código e carregue o dataset desejado. Ao utilizar outro dataset, certifique-se de alterar o nome do arquivo no código:
   ```python
   df = pd.read_csv('/content/nome-do-dataset.csv')
   ```

3. O relatório será gerado e exibido automaticamente na interface do Colab.

### Conclusão

Este projeto demonstra como é possível automatizar a análise exploratória de dados de forma eficiente e visualmente atrativa. O uso de **pandas** para manipulação dos dados, aliado às bibliotecas de visualização e ao uso de templates HTML com **jinja2**, permite a criação de relatórios ricos em informações e acessíveis. A estrutura modular do código facilita a adaptação para diferentes tipos de datasets, tornando esta ferramenta um ponto de partida poderoso para a análise de dados.

Com este relatório automatizado, é possível obter insights rápidos e precisos, facilitando a identificação de oportunidades e desafios dentro dos dados e servindo como base para estudos e modelagens futuras.

In [ ]:
# Instalação de bibliotecas necessárias
!pip install jinja2
!pip install matplotlib seaborn pandas

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from jinja2 import Environment, FileSystemLoader
import os
import base64
from io import BytesIO

# Configuração do ambiente para Jinja2 no Colab
!mkdir -p /content/templates

# Template HTML com Bootstrap e título dinâmico
template_html = """
<!DOCTYPE html>
<html lang="pt-br">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Relatório com Análise dos Dados Automatizada de {{ dataset_name }}</title>
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
    <style>
        .conclusao-item {
            margin-bottom: 15px;
            padding: 10px;
            background-color: #f8f9fa;
            border: 1px solid #dee2e6;
            border-radius: 5px;
        }
        .conclusao-item strong {
            color: #343a40;
        }
    </style>
</head>
<body class="bg-light">
    <div class="container my-5">
        <header class="mb-5">
            <h1 class="text-center mb-4">Relatório com Análise dos Dados Automatizada de {{ dataset_name }}</h1>
            <p class="text-center">Este relatório apresenta uma análise exploratória automatizada dos dados do arquivo <strong>{{ dataset_name }}</strong>. Abaixo, você verá estatísticas, tipos de variáveis e gráficos que ajudam a entender melhor o conjunto de dados.</p>
        </header>

        <section>
            <div class="card mb-4">
                <div class="card-body">
                    <h2 class="card-title">Sumário</h2>
                    <p>O sumário apresenta informações básicas sobre o dataset, proporcionando uma visão geral das características e da qualidade dos dados disponíveis. Veja abaixo o significado de cada item:</p>
                    <ul class="list-group list-group-flush">
                        <li class="list-group-item">
                            <strong>Número de Linhas:</strong> {{ summary.shape[0] }}
                            <br>Refere-se ao total de registros ou entradas no dataset. Cada linha normalmente representa uma observação ou caso específico.</li>
                        <li class="list-group-item">
                            <strong>Número de Colunas:</strong> {{ summary.shape[1] }}
                            <br>Indica quantas variáveis ou atributos estão presentes no dataset. Cada coluna contém um tipo específico de informação sobre as observações.</li>
                        <li class="list-group-item">
                            <strong>Número de Variáveis:</strong> {{ summary.num_var }}
                            <br>O número total de variáveis ou características presentes no dataset. Elas podem ser numéricas, categóricas ou de outros tipos.</li>
                        <li class="list-group-item">
                            <strong>Número de Observações:</strong> {{ summary.num_obs }}
                            <br>Este valor é o mesmo que o número de linhas, indicando quantos exemplos ou casos estão sendo analisados no dataset.</li>
                        <li class="list-group-item">
                            <strong>Células Faltantes:</strong> {{ summary.cels_faltantes }}
                            <br>Refere-se à quantidade de células no dataset que não possuem valores preenchidos. Valores faltantes podem indicar dados ausentes ou erros de coleta e podem influenciar a análise se não forem tratados.</li>
                        <li class="list-group-item">
                            <strong>Linhas Duplicadas:</strong> {{ summary.linhas_duplicadas }}
                            <br>Mostra o número de linhas que aparecem mais de uma vez no dataset. Linhas duplicadas podem ser um sinal de inconsistência ou erro nos dados e devem ser tratadas para garantir a qualidade da análise.</li>
                    </ul>
                </div>
            </div>
        </section>

        <section>
            <div class="card mb-4">
                <div class="card-body">
                    <h3 class="card-title">Tipos de Dados</h3>
                    <p>Esta seção lista os tipos de dados presentes no dataset, permitindo entender a estrutura das variáveis.</p>
                    <ul class="list-group">
                        {% for column, dtype in summary.dtypes.items() %}
                        <li class="list-group-item">{{ column }}: {{ dtype }}</li>
                        {% endfor %}
                    </ul>
                </div>
            </div>
        </section>

        <section>
            <div class="card mb-4">
                <div class="card-body">
                    <h3 class="card-title">Estatísticas Descritivas</h3>
                    <p>A tabela abaixo apresenta as estatísticas descritivas das variáveis numéricas no dataset. Essas estatísticas ajudam a entender a distribuição, a centralidade e a dispersão dos dados.</p>
                    <ul>
                        <li><strong>Média:</strong> A média é a soma de todos os valores dividida pelo número de valores. Ela fornece uma medida do valor central dos dados, mas pode ser influenciada por outliers (valores extremos).</li>
                        <li><strong>Mediana:</strong> A mediana é o valor que separa os dados em duas metades iguais. Ao contrário da média, a mediana não é afetada por valores extremos e é útil para identificar a tendência central em distribuições assimétricas.</li>
                        <li><strong>Desvio Padrão:</strong> O desvio padrão mede a dispersão dos dados em relação à média. Um desvio padrão alto indica que os valores estão mais espalhados, enquanto um desvio padrão baixo indica que os valores estão mais próximos da média.</li>
                        <li><strong>Mínimo:</strong> O valor mínimo é o menor valor presente na variável. Ele ajuda a identificar o limite inferior dos dados e pode indicar a presença de valores extremos baixos.</li>
                        <li><strong>Máximo:</strong> O valor máximo é o maior valor presente na variável. Ele indica o limite superior dos dados e pode destacar valores extremos altos.</li>
                    </ul>
                    <p>Essas estatísticas são essenciais para entender a natureza dos dados, identificar possíveis outliers e analisar a variabilidade dentro das variáveis.</p>
                    <table class="table table-striped">
                        <thead>
                            <tr>
                                <th>Variável</th>
                                <th>Média</th>
                                <th>Mediana</th>
                                <th>Desvio Padrão</th>
                                <th>Mínimo</th>
                                <th>Máximo</th>
                            </tr>
                        </thead>
                        <tbody>
                            {% for column, stats in summary.status_descritiveis.items() %}
                            <tr>
                                <td>{{ column }}</td>
                                <td>{{ stats['mean'] }}</td>
                                <td>{{ stats['50%'] }}</td>
                                <td>{{ stats['std'] }}</td>
                                <td>{{ stats['min'] }}</td>
                                <td>{{ stats['max'] }}</td>
                            </tr>
                            {% endfor %}
                        </tbody>
                    </table>
                </div>
            </div>
        </section>

        <section>
            <h2 class="mt-5">Gráficos Gerados pelos Dados Analisados de {{ dataset_name }}</h2>
            <p>Os gráficos abaixo apresentam uma visão detalhada das variáveis e suas distribuições no dataset. Cada gráfico ajuda a visualizar diferentes aspectos dos dados:</p>
            <ul>
                <li><strong>Histogramas:</strong> Mostram a distribuição das variáveis numéricas, indicando quantas vezes cada valor ou intervalo de valores aparece nos dados. Eles são úteis para entender se os dados estão concentrados em uma faixa específica ou se seguem um padrão, como uma distribuição normal.</li>
                <li><strong>Gráficos de Dispersão:</strong> Utilizados para visualizar a relação entre duas variáveis numéricas. Eles exibem pontos que representam as observações, ajudando a identificar padrões, como uma possível correlação entre as variáveis (por exemplo, quanto uma variável aumenta ou diminui em relação à outra).</li>
                <li><strong>Regressão Linear:</strong> Este gráfico mostra uma linha de tendência adicionada ao gráfico de dispersão. A linha de regressão ajuda a visualizar e quantificar a relação linear entre duas variáveis, indicando se, conforme uma variável aumenta, a outra também tende a aumentar (ou diminuir).</li>
                <li><strong>Boxplots:</strong> Apresentam um resumo visual dos dados numéricos, incluindo a mediana, os quartis e os outliers (valores extremos). Eles são úteis para entender a dispersão dos dados e identificar possíveis valores atípicos que se destacam do restante das observações.</li>
                <li><strong>Mapas de Calor:</strong> Mostram as correlações entre as variáveis numéricas em uma matriz. Cada célula representa a força da correlação entre duas variáveis, usando cores para indicar se a relação é positiva, negativa ou inexistente. São importantes para identificar variáveis que podem ter uma relação significativa, ajudando na escolha de variáveis para futuras análises ou modelos preditivos.</li>
            </ul>
            <div class="row">
                {% for plot in plots %}
                <div class="col-md-6 mb-4">
                    <div class="card">
                        <img src="data:image/png;base64,{{ plot }}" class="card-img-top" alt="Plot">
                    </div>
                </div>
                {% endfor %}
            </div>
        </section>

        <section class="mt-5">
            <div class="card">
                <div class="card-body">
                    <h3 class="card-title">Conclusões das Análises</h3>
                    <div>
                        {% for item in conclusao %}
                        <div class="conclusao-item">
                            <p>{{ item }}</p>
                        </div>
                        {% endfor %}
                    </div>
                </div>
            </div>
        </section>

        <section class="mt-5">
          <div class="card">
              <div class="card-body">
                  <h3 class="card-title">Considerações Finais</h3>
                  <p>Este projeto teve como objetivo realizar uma análise exploratória automatizada de um conjunto de dados utilizando métodos estatísticos e gráficos para identificar padrões, correlações e anomalias presentes no dataset <strong>{{ dataset_name }}</strong>. O relatório gerado é uma ferramenta que permite uma visão detalhada e clara das características do dataset, facilitando a tomada de decisões e o planejamento de próximas etapas na análise de dados.</p>
                  <p>Inicialmente, foram coletadas informações básicas sobre o conjunto de dados, como o número de linhas, colunas, variáveis e observações. Em seguida, exploramos os tipos de variáveis presentes (numéricas e categóricas) e identificamos valores ausentes, além de linhas duplicadas que poderiam impactar a qualidade das análises.</p>
                  <p>O relatório também apresentou uma análise detalhada através de estatísticas descritivas, que mostraram medidas como média, mediana, desvio padrão, valores mínimos e máximos para cada variável numérica. Essas informações são essenciais para entender a distribuição e a variabilidade dos dados, o que é crucial para futuras modelagens e predições.</p>
                  <p>Adicionalmente, gráficos foram gerados para visualizar as distribuições das variáveis, suas correlações e outros padrões visuais que poderiam indicar a presença de outliers ou tendências específicas. Histogramas, gráficos de dispersão, regressão linear, boxplots e mapas de calor foram utilizados para representar visualmente as características mais importantes do conjunto de dados.</p>
                  <p>Ao longo do processo, identificamos variáveis com outliers significativos, distribuições assimétricas e fortes correlações entre algumas variáveis. Esses achados são fundamentais para ajustar e preparar os dados para futuras análises mais complexas, como a aplicação de algoritmos de machine learning ou a realização de modelagens preditivas.</p>
                  <p>Em resumo, o relatório gerado proporciona uma visão abrangente e detalhada do dataset <strong>{{ dataset_name }}</strong>, servindo como um ponto de partida para análises mais profundas e refinadas. Ao automatizar o processo de análise exploratória, conseguimos obter insights rápidos e precisos, o que facilita a identificação de oportunidades e desafios dentro dos dados. É recomendado que, com base nos achados deste relatório, sejam realizados ajustes e transformações nos dados para otimizar seu uso em estudos futuros.</p>
              </div>
          </div>
        </section>

        <footer class="mt-5">
            <p class="text-center">Relatório gerado automaticamente por nosso sistema de análise. Para mais informações, entre em contato com a equipe de desenvolvimento.</p>
        </footer>
    </div>

    <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/@popperjs/core@2.9.2/dist/umd/popper.min.js"></script>
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>
</body>
</html>
"""

# Salvando o template HTML no sistema de arquivos do Colab
with open('/content/templates/report_template.html', 'w') as file:
    file.write(template_html)

def resumo_df(df):
    num_var = len(df.columns)
    num_obs = len(df)
    cels_faltantes = df.isnull().sum().sum()
    linhas_duplicadas = df.duplicated().sum()

    numeric_vars = len(df.select_dtypes(include=np.number).columns)
    categorical_vars = len(df.select_dtypes(include='object').columns)

    summary = {
        'shape': df.shape,
        'num_var': num_var,
        'num_obs': num_obs,
        'cels_faltantes': cels_faltantes,
        'linhas_duplicadas': linhas_duplicadas,
        'numeric_vars': numeric_vars,
        'categorical_vars': categorical_vars,
        'dtypes': df.dtypes.to_dict(),
        'valores_faltantes': df.isnull().sum().to_dict(),
        'duplicados': df.duplicated().sum(),
        'status_descritiveis': df.describe().to_dict()
    }
    return summary

# Função para gerar conclusões com base nos dados
def gerar_conclusao(df):
    conclusoes = []

    # Analisar correlações
    numeric_columns = df.select_dtypes(include=np.number).columns
    if len(numeric_columns) > 1:
        correlation_matrix = df[numeric_columns].corr()
        correlated_pairs = correlation_matrix[(correlation_matrix > 0.7) | (correlation_matrix < -0.7)].stack().reset_index()
        correlated_pairs = correlated_pairs[correlated_pairs['level_0'] != correlated_pairs['level_1']]

        if not correlated_pairs.empty:
            for _, row in correlated_pairs.iterrows():
                conclusoes.append(f"As variáveis '{row['level_0']}' e '{row['level_1']}' apresentam uma correlação de {row[0]:.2f}, o que pode indicar uma relação significativa.")

    # Analisar outliers nas variáveis numéricas
    for column in numeric_columns:
        if ((df[column] < df[column].quantile(0.25) - 1.5 * (df[column].quantile(0.75) - df[column].quantile(0.25))) |
            (df[column] > df[column].quantile(0.75) + 1.5 * (df[column].quantile(0.75) - df[column].quantile(0.25)))).any():
            conclusoes.append(f"A variável '{column}' apresenta outliers significativos.")

    # Analisar valores faltantes
    high_missing = df.isnull().mean() > 0.2
    missing_cols = df.columns[high_missing]
    for col in missing_cols:
        conclusoes.append(f"A coluna '{col}' possui mais de 20% de valores faltantes, o que pode impactar a análise se não for tratado.")

    # Analisar distribuições das variáveis numéricas
    for column in numeric_columns:
        skewness = df[column].skew()
        if skewness > 1:
            conclusoes.append(f"A variável '{column}' apresenta uma distribuição assimétrica positiva, indicando que a maioria dos valores está concentrada à esquerda.")
        elif skewness < -1:
            conclusoes.append(f"A variável '{column}' apresenta uma distribuição assimétrica negativa, indicando que a maioria dos valores está concentrada à direita.")

    if not conclusoes:
        conclusoes.append("Nenhum padrão ou correlação significativa foi identificada nos dados.")

    return conclusoes

# Função para converter gráficos para Base64
def plot_64(fig):
    buf = BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    buf.close()
    return img_base64

# Função para gerar gráficos e retornar como Base64
def criar_plot(df):
    plots = []

    # Histogramas para variáveis numéricas
    for column in df.select_dtypes(include=np.number).columns:
        fig, ax = plt.subplots()
        sns.histplot(df[column], kde=True, ax=ax)
        plt.title(f'Histograma de {column}')
        img_base64 = plot_64(fig)
        plots.append(img_base64)
        plt.close(fig)

    # Gráficos de dispersão para variáveis numéricas (se houver mais de uma)
    numeric_columns = df.select_dtypes(include=np.number).columns
    if len(numeric_columns) > 1:
        fig, ax = plt.subplots()
        sns.scatterplot(data=df, x=numeric_columns[0], y=numeric_columns[1], ax=ax)
        plt.title('Gráfico de Dispersão')
        img_base64 = plot_64(fig)
        plots.append(img_base64)
        plt.close(fig)

        # Gráfico de regressão linear entre as duas primeiras variáveis numéricas
        fig, ax = plt.subplots()
        sns.regplot(x=numeric_columns[0], y=numeric_columns[1], data=df, ax=ax, line_kws={"color": "red"})
        plt.title(f'Regressão Linear entre {numeric_columns[0]} e {numeric_columns[1]}')
        img_base64 = plot_64(fig)
        plots.append(img_base64)
        plt.close(fig)

    # Boxplot para variáveis numéricas
    for column in df.select_dtypes(include=np.number).columns:
        fig, ax = plt.subplots()
        sns.boxplot(data=df[column], ax=ax)
        plt.title(f'Boxplot de {column}')
        img_base64 = plot_64(fig)
        plots.append(img_base64)
        plt.close(fig)

    # Gráfico de linha para as primeiras 10 observações de uma coluna numérica
    for column in df.select_dtypes(include=np.number).columns:
        fig, ax = plt.subplots()
        df[column].head(10).plot(kind='line', ax=ax)
        plt.title(f'Gráfico de Linha para {column} (primeiras 10 observações)')
        img_base64 = plot_64(fig)
        plots.append(img_base64)
        plt.close(fig)

    # Mapa de calor de correlação
    if len(numeric_columns) > 1:
        fig, ax = plt.subplots()
        correlation = df[numeric_columns].corr()
        sns.heatmap(correlation, annot=False, cmap='coolwarm', ax=ax)
        plt.title('Mapa de Calor da Correlação')
        img_base64 = plot_64(fig)
        plots.append(img_base64)
        plt.close(fig)

    return plots

# Função para gerar relatório HTML
def gerar_html(df, file_name):
    env = Environment(loader=FileSystemLoader('/content/templates'))
    template = env.get_template('report_template.html')

    summary = resumo_df(df)
    conclusao = gerar_conclusao(df)
    plots = criar_plot(df)

    html_content = template.render(summary=summary, plots=plots, dataset_name=file_name, conclusao=conclusao)

    # Salvar o relatório HTML no Colab
    report_path = '/content/relatorio.html'
    with open(report_path, 'w') as f:
        f.write(html_content)

    print(f'Report saved at: {report_path}')
    return report_path

if __name__ == '__main__':
    file_name = 'Diabetes.csv'
    df = pd.read_csv(f'/content/{file_name}')
    report_path = gerar_html(df, file_name)

    # Exibir o relatório HTML no Colab
    from IPython.display import IFrame
    IFrame(report_path, width=800, height=600)



Report saved at: /content/relatorio.html
